# Photometric data of directly imaged planets

In this tutorial, we will add photometric data, distances, and spectra of directly imaged planets and brown dwarfs to the database and use the reading functionalities to easily access those data.

## Getting started

We start by importing the required Python packages.

In [1]:
import species

The [species](https://species.readthedocs.io/en/latest/species.html) HDF5 database is initiated by creating an instance of the [SpeciesInit](https://species.readthedocs.io/en/latest/species.core.html#species.core.init.SpeciesInit) class.

In [2]:
species.SpeciesInit()

Initiating species v0.4.0... [DONE]
Creating species_config.ini... [DONE]
Database: /Users/tomasstolker/applications/species/docs/tutorials/species_database.hdf5
Data folder: /Users/tomasstolker/applications/species/docs/tutorials/data
Working folder: /Users/tomasstolker/applications/species/docs/tutorials
Creating species_database.hdf5... [DONE]
Creating data folder... [DONE]


## Adding companion data to the database

To add data to the database, we first create an instance of [Database](https://species.readthedocs.io/en/latest/species.data.html#species.data.database.Database).

In [3]:
database = species.Database()

Next, we use the [add_companion](https://species.readthedocs.io/en/latest/species.data.html#species.data.database.Database.add_companion) method of `Database` to add all companion data that is [available in the species toolkit](https://github.com/tomasstolker/species/blob/main/species/data/companions.py). This includes filter names, magnitudes, and distances of most directly imaged planets and brown dwarfs, as well as spectra for some objects. We set the argument of `name` to `None` such that all data are added instead of selecting a single companion.

Running this method will also download the relevant filter profiles from the [SVO Filter Profile Service](http://svo2.cab.inta-csic.es/svo/theory/fps/) and a flux-calibrated spectrum of Vega. The magnitudes that are available in the toolkit are converted into fluxes and both stored in the database. We set `verbose` to `False` to not print a overview of all the filter and companion data that are being added.

In [4]:
database.add_companion(name=None, verbose=False)

Adding Vega spectrum... [DONE]


/Users/tomasstolker/applications/species/species/data/filters.py:210: UserWarning: The minimum transmission value of Subaru/CIAO.z is smaller than zero (-1.80e-03). Wavelengths with negative transmission values will be removed.
  warnings.warn(


To get an overview of the companion data that are stored in the database, one can use the [list_companions](https://species.readthedocs.io/en/latest/species.data.html#species.data.database.Database.list_companions) method of `Database`. This will also return a list with the companion names, which could for example be iterated when using the reading functionalities below.

## Reading companion data

For reading data of individual companions/objects, we can use the reading functionalities of the toolkit. We start by creating an instance of [ReadObject](https://species.readthedocs.io/en/latest/species.read.html#species.read.read_object.ReadObject) with the object name as stored in the database given as argument.

In [5]:
read_obj = species.ReadObject('beta Pic b')

We can  check for which filters there is photometric data available with the [list_filters](https://species.readthedocs.io/en/latest/species.read.html#species.read.read_object.ReadObject.list_filters) method of `ReadObject`.

In [6]:
filter_list = read_obj.list_filters()

Available photometric data for beta Pic b:
   - Gemini/NICI.ED286
   - Magellan/VisAO.Ys
   - Paranal/NACO.H
   - Paranal/NACO.J
   - Paranal/NACO.Ks
   - Paranal/NACO.Lp
   - Paranal/NACO.Mp
   - Paranal/NACO.NB374
   - Paranal/NACO.NB405


In [7]:
print(filter_list)

['Gemini/NICI.ED286', 'Magellan/VisAO.Ys', 'Paranal/NACO.H', 'Paranal/NACO.J', 'Paranal/NACO.Ks', 'Paranal/NACO.Lp', 'Paranal/NACO.Mp', 'Paranal/NACO.NB374', 'Paranal/NACO.NB405']


Let's now extract the photometric data of the NACO $M'$ filter. The [get_photometry](https://species.readthedocs.io/en/latest/species.read.html#species.read.read_object.ReadObject.get_photometry) method returns an array with the apparent magnitude, magnitude error, flux density (W m$^{-2}$ $\mu$m$^{-1}$), and flux density error.

In [8]:
mp_phot = read_obj.get_photometry('Paranal/NACO.Mp')

Similarly, we can use the [get_spectrum](https://species.readthedocs.io/en/latest/species.read.html#species.read.read_object.get_spectrum) method to return a dictionary with available spectra. In this case a GPI spectrum in the $YJHK$ bands from [Chilcote et al. 2017](https://ui.adsabs.harvard.edu/abs/2017AJ....153..182C/abstract) and a GRAVITY spectrum in the $K$ band from [Gravity Collaboration et al. 2020](https://ui.adsabs.harvard.edu/abs/2020A%26A...633A.110G/abstract).

In [9]:
spec_dict = read_obj.get_spectrum()
print(spec_dict.keys())

dict_keys(['GPI_YJHK', 'GRAVITY'])


Other methods for `ReadObject` are [get_distance](https://species.readthedocs.io/en/latest/species.read.html#species.read.read_object.ReadObject.get_distance) for returning the distance and [get_absmag](https://species.readthedocs.io/en/latest/species.read.html#species.read.read_object.ReadObject.get_absmag) for calculating the absolute magnitude and uncertainty.

In [10]:
distance = read_obj.get_distance()
print(f'Distance (pc) = {distance[0]:.2f} +/- {distance[1]:.2f}')

Distance (pc) = 19.75 +/- 0.13


In [11]:
abs_mag = read_obj.get_absmag('Paranal/NACO.Mp')
print(f'Apparent magnitude = {mp_phot[0]:.2f} +/- {mp_phot[1]:.2f}')
print(f'Absolute magnitude = {abs_mag[0]:.2f} +/- {abs_mag[1]:.2f}')

Apparent magnitude = 11.10 +/- 0.12
Absolute magnitude = 9.62 +/- 0.12


## Extracting an ObjectBox with data

Instead of using the `ReadObject` functionalities, we can also use the [get_object](https://species.readthedocs.io/en/latest/species.data.html#species.data.database.Database.get_object) method of `Database` to extract an [ObjectBox](https://species.readthedocs.io/en/latest/species.core.html#species.core.box.ObjectBox) with the companion data from the database. The `inc_phot` and `inc_spec` parameters are either a boolean or a list of filter names / spectra. In this example, we will include all photometric data.

In [12]:
object_box = database.get_object('beta Pic b', inc_phot=True, inc_spec=False)

Getting object: beta Pic b... [DONE]


We can inspect the content of the `ObjectBox` with the [open_box](https://species.readthedocs.io/en/latest/species.core.html#species.core.box.Box.open_box) method. Let's have a look!

In [13]:
object_box.open_box()

Opening ObjectBox...
name = beta Pic b
filters = ['Gemini/NICI.ED286', 'Magellan/VisAO.Ys', 'Paranal/NACO.H', 'Paranal/NACO.J', 'Paranal/NACO.Ks', 'Paranal/NACO.Lp', 'Paranal/NACO.Mp', 'Paranal/NACO.NB374', 'Paranal/NACO.NB405']
mean_wavel = {'Gemini/NICI.ED286': 1.5841804, 'Magellan/VisAO.Ys': 0.9826820974261752, 'Paranal/NACO.H': 1.6588092, 'Paranal/NACO.J': 1.2650998, 'Paranal/NACO.Ks': 2.1449544, 'Paranal/NACO.Lp': 3.8050284, 'Paranal/NACO.Mp': 4.780971, 'Paranal/NACO.NB374': 3.744805, 'Paranal/NACO.NB405': 4.0558634}
magnitude = {'Gemini/NICI.ED286': array([13.18,  0.15]), 'Magellan/VisAO.Ys': array([15.53,  0.34]), 'Paranal/NACO.H': array([13.32,  0.14]), 'Paranal/NACO.J': array([14.11,  0.21]), 'Paranal/NACO.Ks': array([12.64,  0.11]), 'Paranal/NACO.Lp': array([11.3 ,  0.06]), 'Paranal/NACO.Mp': array([11.1 ,  0.12]), 'Paranal/NACO.NB374': array([11.25,  0.23]), 'Paranal/NACO.NB405': array([10.98,  0.05])}
flux = {'Gemini/NICI.ED286': array([6.98948226e-15, 9.68707404e-16]), 'Ma

Each [Box](https://species.readthedocs.io/en/latest/species.core.html#species.core.box.Box) is a Python object and the items in a box can be extracted as attributes. For example, to get the list of filter names:

In [14]:
print(object_box.filters)

['Gemini/NICI.ED286', 'Magellan/VisAO.Ys', 'Paranal/NACO.H', 'Paranal/NACO.J', 'Paranal/NACO.Ks', 'Paranal/NACO.Lp', 'Paranal/NACO.Mp', 'Paranal/NACO.NB374', 'Paranal/NACO.NB405']
